In [1]:
# importamos las librerías que necesitamos

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Visualización
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Evaluar linealidad de las relaciones entre las variables
# y la distribución de las variables
# ------------------------------------------------------------------------------
import scipy.stats as stats
from scipy.stats import chi2_contingency, ttest_ind

# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

# Gestión de los warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")

In [5]:
df = pd.read_csv("Files/marketing_AB.csv", index_col=0)
df.head()

,user id,test group,converted,total ads,most ads day,most ads hour
0,1069124,ad,False,130,Monday,20
1,1119715,ad,False,93,Tuesday,22
2,1144181,ad,False,21,Tuesday,18
3,1435133,ad,False,355,Tuesday,10
4,1015700,ad,False,276,Friday,14


Supongamos que estamos realizando un experimento para determinar si un nuevo diseño de anuncio (test group "ad") tiene un impacto significativo en la tasa de conversión en comparación con el diseño de anuncio anterior (test group "psa").
- Objetivo del ejercicio: Comparar las tasas de conversión entre los grupos de prueba "ad" y "psa" para determinar si el nuevo diseño de anuncio es más efectivo en la conversión de usuarios.

Preparación de los datos: Asegúrate de que los datos estén limpios y listos para el análisis. Esto incluye la eliminación de datos faltantes y la verificación de la coherencia de los datos.

In [8]:
df.shape

(588101, 6)

In [6]:
df.info()
# el conjunto de datos no tiene nulos

<class 'pandas.core.frame.DataFrame'>
Index: 588101 entries, 0 to 588100
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   user id        588101 non-null  int64 
 1   test group     588101 non-null  object
 2   converted      588101 non-null  bool  
 3   total ads      588101 non-null  int64 
 4   most ads day   588101 non-null  object
 5   most ads hour  588101 non-null  int64 
dtypes: bool(1), int64(3), object(2)
memory usage: 27.5+ MB


In [10]:
df['test group'].unique()
# los valores son correctos y completos

array(['ad', 'psa'], dtype=object)

In [11]:
df['converted'].unique()
# es un valor booleano, los datos están completos y correctos

array([False,  True])

In [12]:
df['most ads day'].unique()
# los datos están completos y correctos

array(['Monday', 'Tuesday', 'Friday', 'Saturday', 'Wednesday', 'Sunday',
       'Thursday'], dtype=object)

In [21]:
type(df.describe())

pandas.core.frame.DataFrame

In [25]:
df.describe().loc[:, ['total ads','most ads hour']]

,total ads,most ads hour
count,588101.000000,588101.000000
mean,24.820876,14.469061
std,43.715181,4.834634
min,1.000000,0.000000
25%,4.000000,11.000000
50%,13.000000,14.000000
75%,27.000000,18.000000
max,2065.000000,23.000000


In [27]:
df.sample(5)

,user id,test group,converted,total ads,most ads day,most ads hour
391011,1099841,ad,False,9,Saturday,12
161786,1418599,ad,True,162,Thursday,15
452780,1104952,ad,False,1,Monday,21
147851,1586921,ad,False,41,Tuesday,21
564150,1221998,ad,False,2,Thursday,8


Definición de las hipótesis: Plantea una hipótesis nula (H0) y una hipótesis alternativa (H1) para el experimento (en este caso os las damos definidas):

- Hipótesis nula (H0): No hay diferencia significativa en la tasa de conversión entre los grupos de prueba "ad" y "psa".

- Hipótesis alternativa (H1): Existe una diferencia significativa en la tasa de conversión entre los grupos de prueba "ad" y "psa".

In [ ]:
# Procedemos a hacer las pruebas

# Primero analizamos visualmente:

sns.barplot(x="group_ad", y="total ads", data=df)


sns.barplot(x="group_psa", y="total ads", data=df)

In [ ]:
df['test group'] == 'ad' & df['converted'] == True

In [50]:
filtro_ad = df[(df['test group'] == 'ad') & (df['converted'] == True)]

In [52]:
filtro_psa = df[(df['test group'] == 'psa') & (df['converted'] == True)]

In [51]:
filtro_ad

,user id,test group,converted,total ads,most ads day,most ads hour,ad
15,1461774,ad,True,9,Wednesday,18,65344.555556
44,1355531,ad,True,265,Tuesday,12,2219.249057
107,1389878,ad,True,1328,Saturday,19,442.847139
121,1475989,ad,True,323,Saturday,20,1820.746130
135,1241733,ad,True,246,Friday,20,2390.654472
...,...,...,...,...,...,...,...
586037,1473422,ad,True,5,Tuesday,13,117620.200000
586073,1259895,ad,True,24,Tuesday,14,24504.208333
586343,1371997,ad,True,14,Tuesday,17,42007.214286
587069,1586485,ad,True,4,Tuesday,16,147025.250000


In [53]:
filtro_psa

,user id,test group,converted,total ads,most ads day,most ads hour,ad
957,915026,psa,True,88,Sunday,19,6682.965909
2531,904434,psa,True,7,Tuesday,20,84014.428571
2744,920737,psa,True,77,Monday,13,7637.675325
2751,904131,psa,True,111,Monday,12,5298.207207
2979,901749,psa,True,364,Friday,21,1615.662088
...,...,...,...,...,...,...,...
569821,902040,psa,True,13,Friday,19,45238.538462
577022,917817,psa,True,7,Sunday,13,84014.428571
578515,913311,psa,True,2,Sunday,19,294050.500000
586818,913022,psa,True,11,Tuesday,15,53463.727273


In [64]:
filtro1 = df[df['converted'] == True]

In [68]:
filtro1.shape

(14843, 7)

In [63]:
(df['converted'] == True).count()

588101

In [54]:
# Tasas de conversión por grupos

conversion_rate_ad = filtro_ad['user id'].count() / df.shape[0]
conversion_rate_psa = filtro_psa['user id'].count() / df.shape[0]


In [56]:
conversion_rate_ad

0.02452469898877914

In [55]:
conversion_rate_psa

0.0007141630434228134

Prueba de hipótesis: Utiliza una prueba estadística, como la prueba t de Student o la prueba chi-cuadrado, para comparar las tasas de conversión entre los grupos "ad" y "psa".

In [72]:
group_ad = df[df['test group'] == 'ad']
group_psa = df[df['test group'] == 'psa']

# Realizar la prueba de hipótesis (prueba t de Student)
# Buscamos si hay diferencia significativa entre los dos ratios de conversión de estos dos grupos.
t_stat, p_value = stats.ttest_ind(group_ad['converted'], group_psa['converted'])

# Definir un nivel de significancia (alfa)
alfa = 0.05

# Imprimir los resultados
print(f"Tasa de conversión en el grupo 'ad': {conversion_rate_ad:.2f}")
print(f"Tasa de conversión en el grupo 'psa': {conversion_rate_psa:.2f}")
print(f"Valor p de la prueba de hipótesis: {p_value:.12f}")

# Hay una diferencia significativa entre las tasas de conversión entre los grupos, concluyendo que el grupo AD, que es nuestro grupo de control, 
# ha tenido un ratio de conversión mucho más grande que el del grupo de prueba, es decir, 
# el diseño del nuevo anuncio, NO es más efectivo en la conversión de usuarios con respecto al diseño anterior.
# Nuestra recomendación es que el diseño de la web quede como estaba anteriormente. 
# Dado que se demuestra que el diseño anterior es más efectivo en la conversión de usuarios. 

Tasa de conversión en el grupo 'ad': 0.02
Tasa de conversión en el grupo 'psa': 0.00
Valor p de la prueba de hipótesis: 0.000000000000


Análisis de resultados:

- Si el valor p es menor que un umbral predefinido (por ejemplo, 0.05), rechazamos la hipótesis nula y concluimos que hay una diferencia significativa en las tasas de conversión entre los grupos.

- Si el valor p es mayor que el umbral, no podemos rechazar la hipótesis nula y concluimos que no hay evidencia suficiente para afirmar que hay una diferencia significativa.

Conclusiones y recomendaciones: Basándote en los resultados de la prueba de hipótesis, llega a una conclusión sobre si el nuevo diseño de anuncio es más efectivo en la conversión de usuarios que el diseño anterior. Si es así, puedes hacer recomendaciones basadas en estos hallazgos.